# Edge detection

This Notebook illustrates how to use the Kymotracker in Pylake to detect edges on a kymograph. In the example data, ssDNA is coated with RPA. A T7 Pol is performing exonucleolyis, thus increasing the region of ssDNA. This is seen as a wideining of the RPA region, in Blue. The aim is to find the speed of this widening, by detecting the edge of the RPA coated region.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import lumicks.pylake as lk
from skimage import filters

%matplotlib widget

linear_colormaps = {
"red": LinearSegmentedColormap.from_list("red", colors=[(0, 0, 0), (1, 0, 0)]),
"green": LinearSegmentedColormap.from_list("green", colors=[(0, 0, 0), (0, 1, 0)]),
"blue": LinearSegmentedColormap.from_list("blue", colors=[(0, 0, 0), (0, 0, 1)]),
}

lk.__version__

In [ ]:
f = lk.File('Kymograph_bp7_t1_pol_rpa_ext_short_nick_no_hf.h5')

### Plot the kymograph

In [ ]:
name, kymo = f.kymos.popitem()

plt.figure()
kymo.plot(channel = 'blue', vmax=4, aspect = 5) 

The activity of T7 Pol depends on force. For high force, exonucleolysis dominates, while for low force polymerization dominates. In the kymograph, you can see the blue, RPA coated region increase and decrease over time as the force is increased and decreased respectively. Below, we select a region of interest for further analysis.

In [ ]:
kymo_slice=kymo['32s':'65s'] # Select the time range for which you want to compute the speeds
plt.figure()
kymo_slice.plot(channel = 'blue', vmax=4) # Adjust contrast here by changing the value of 'vmax'

### Crop kymo to remove beads

Remove the beads from the kymo by choosing the coordinates of the bead edges. 

In [ ]:
plt.figure()
beadedge_top=3.4
beadedge_bot= 5.8
cropped_kymo = kymo_slice.crop_by_distance(beadedge_top,beadedge_bot)
cropped_kymo.plot(channel = 'blue', vmax=4, aspect = 5)

### Edge filter

Below a so-called sobel image filter is applied, which highlights the edges in an image.
A directional filter is applied, such that horizontal edges are emphasized, and vertical edges are ignored.

In [ ]:
image = cropped_kymo.get_image('blue')
edge_filter = filters.sobel(image, axis=[0,0,0])

plt.figure(figsize=(10,6))
plt.imshow(edge_filter)
plt.axis('tight')
plt.tight_layout()
plt.show()

In the image above, the two lines actually corerspond to the edges of the RPA region. To detect those edges, we are going to use the Kymotracker.

### Create kymo image

In the current version of Pylake (1.2), the Kymotracker only works on Pylake kymo objects. Therefore the image of the kymo has to be converted back to a kymo object.

In [ ]:
mock = lk.kymo._kymo_from_array(edge_filter, 'b', kymo.line_time_seconds, pixel_size_um=1.0)

## Use Kymotracker to detect edges

Below, we are going to load the image with the highlighted edges as shown above into the kymotracker. Load the kymotracker and click 'Track all'. Adjust the tracking parameter and remove, connect or split tracks if needed. Then save the coordinates of the edges by clicking 'save.'

In [ ]:
plt.close('all')

kw_blue = lk.KymoWidgetGreedy(mock, "blue", pixel_threshold=3, axis_aspect_ratio=0.7, min_length=10, window=6, sigma=0.20, vmax=4, track_width = 5,
    slider_ranges={"sigma": (0.1, 1)}, correct_origin = True)

### Overlay tracked edges with original image

The edge coordinates can be plotted on top of the original kymograph. Load the kymotracker with the original kymograph below, and click 'load' to load the edge coordinates that you just tracked.

In [ ]:
kw_blue_original = lk.KymoWidgetGreedy(cropped_kymo, "blue",vmax=5, aspect = 2)

## Further analysis.

The tracked edges can now be used for further analysis, for example to determine the speed of the edge.

In [ ]:
speeds=np.array([])

plt.figure(figsize=(15,10))
i=1

for line in kw_blue.tracks:
    plt.subplot(5,6,i)
    sec = line.seconds
    pos = line.position*cropped_kymo.pixelsize_um[0] # Convert pixel position to um
    lin_fit=np.polyfit(sec, pos, 1)
    lin_func=np.poly1d(lin_fit)
    plt.plot(sec,pos,label='data')
    v=lin_fit[0]
    plt.plot(sec,lin_func(sec),label=f'v={v:0.2f} um/s')
    speeds=np.append(speeds,v)
    i+=1
    plt.legend()

plt.tight_layout()
plt.show()
# -

In [ ]:
plt.figure()
plt.hist(speeds)
plt.xlabel('Speed (um/s)')
plt.show()

In [ ]:
np.savetxt('spees_kymo_'+name +'.txt',speeds)